In [1]:
import pandas as pd
import numpy as np  
from datetime import datetime
import math
from pandas.tseries.offsets import Week

In [2]:
trip = pd.read_csv("data/trip.csv")
stations = pd.read_csv("data/station.csv")
weather = pd.read_csv("data/weather.csv")

In [3]:
trip.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103


In [4]:
trip = trip.merge(stations, how="left", left_on = trip.start_station_id, right_on= stations.id)
trip = trip[trip['zip_code'].notna()]
trip = trip[trip.zip_code.str.len()==5]
trip = trip[trip.zip_code.str.startswith('9')]

In [5]:
#remove the trips whose duration is longer than 5 hours and less than 1 min
trip = trip[trip.duration <= 18000]
trip = trip[trip.duration >= 60]

In [6]:
trip.start_date = pd.to_datetime(trip.start_date, format='%m/%d/%Y %H:%M')
trip['date'] = trip.start_date.dt.date


In [7]:
dates = dict()
for i, row in trip.iterrows():
    temp = (row["date"], row["city"])
    if temp not in dates:
        dates[temp] = 1
    else:
        dates[temp] += 1

In [8]:
trip2 = pd.DataFrame.from_dict(dates, orient = "index")
trip2['date'] = list(zip(*trip2.index))[0]
trip2['city'] = list(zip(*trip2.index))[1]
trip2['trips'] = trip2.loc[:,0]
training_data = trip2.loc[:,"date":"trips"]

In [9]:
training_data

,date,city,trips
"(2013-08-29, San Francisco)",2013-08-29,San Francisco,518
"(2013-08-29, San Jose)",2013-08-29,San Jose,58
"(2013-08-29, Mountain View)",2013-08-29,Mountain View,10
"(2013-08-29, Redwood City)",2013-08-29,Redwood City,9
"(2013-08-29, Palo Alto)",2013-08-29,Palo Alto,19
...,...,...,...
"(2014-09-02, Redwood City)",2014-09-02,Redwood City,2
"(2014-09-01, San Francisco)",2014-09-01,San Francisco,216
"(2014-09-01, San Jose)",2014-09-01,San Jose,9
"(2014-09-01, Mountain View)",2014-09-01,Mountain View,11


In [10]:
training_data = training_data.sort_values('date')
training_data.date = pd.to_datetime(training_data.date, format='%Y/%m/%d')
training_data.city = training_data.city.astype('string')

In [11]:
stations.head()

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


In [12]:
stations.installation_date=pd.to_datetime(stations.installation_date, format='%m/%d/%Y')

In [13]:
total_docks = []
i = 0
for index in training_data.index:
    sub_station = stations[stations.city==index[1]]
    total_docks.append(sum(sub_station[sub_station.installation_date <= pd.to_datetime(index[0], format='%Y/%m/%d')].dock_count))

In [14]:
training_data['total_docks'] = total_docks

In [15]:
weather.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,8/30/2013,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0,2.0,NaN,291.0,94107
2,8/31/2013,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0,4.0,NaN,284.0,94107
3,9/1/2013,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0,4.0,NaN,284.0,94107
4,9/2/2013,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0,6.0,NaN,277.0,94107


In [16]:
zip_to_city={95113:"San Jose", 94063:"Redwood City", 94041:"Mountain View", 94301:"Palo Alto", 94107:"San Francisco"}

In [17]:
weather.date = pd.to_datetime(weather.date, format='%m/%d/%Y')
weather.replace(zip_to_city, inplace=True)
weather['zip_code']=weather['zip_code'].astype('string')
weather.rename(columns={'zip_code':'city'}, inplace=True)
weather.isnull().sum()

date                                 0
max_temperature_f                    4
mean_temperature_f                   4
min_temperature_f                    4
max_dew_point_f                     54
mean_dew_point_f                    54
min_dew_point_f                     54
max_humidity                        54
mean_humidity                       54
min_humidity                        54
max_sea_level_pressure_inches        1
mean_sea_level_pressure_inches       1
min_sea_level_pressure_inches        1
max_visibility_miles                13
mean_visibility_miles               13
min_visibility_miles                13
max_wind_Speed_mph                   1
mean_wind_speed_mph                  1
max_gust_speed_mph                 899
precipitation_inches                 1
cloud_cover                          1
events                            3143
wind_dir_degrees                     1
city                                 0
dtype: int64

In [18]:
training_data['year'] = pd.to_datetime(training_data['date']).dt.year
training_data['month'] = pd.to_datetime(training_data['date']).dt.month
training_data['weekday'] = pd.to_datetime(training_data['date']).dt.weekday
training_data.head()

,date,city,trips,total_docks,year,month,weekday
"(2013-08-29, San Francisco)",2013-08-29,San Francisco,518,650,2013,8,3
"(2013-08-29, San Jose)",2013-08-29,San Jose,58,234,2013,8,3
"(2013-08-29, Mountain View)",2013-08-29,Mountain View,10,91,2013,8,3
"(2013-08-29, Redwood City)",2013-08-29,Redwood City,9,100,2013,8,3
"(2013-08-29, Palo Alto)",2013-08-29,Palo Alto,19,75,2013,8,3


In [19]:
weather.events.unique()

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'rain', 'Rain-Thunderstorm'],
      dtype=object)

In [20]:
weather.loc[weather.events == 'rain', 'events'] = "Rain"
weather.loc[weather.events.isnull(), 'events'] = "Normal"

In [21]:
events = pd.get_dummies(weather.events)
weather = weather.merge(events, left_index = True, right_index = True)
#Remove features we don't need
weather = weather.drop(['events'],1)

#max_wind and max_gust are well correlated, so we can use max_wind to help fill the null values of max_gust
#For each value of max_wind, find the median max_gust and use that to fill the null values.
weather.loc[weather.max_gust_speed_mph.isnull(), 'max_gust_speed_mph'] = weather.groupby('max_wind_Speed_mph').max_gust_speed_mph.apply(lambda x: x.fillna(x.median()))

In [22]:
#Change this feature from a string to numeric.
weather.precipitation_inches = pd.to_numeric(weather.precipitation_inches, errors = 'coerce')

In [23]:
#Change null values to the median, of values > 0.
weather.loc[weather.precipitation_inches.isnull(), 
            'precipitation_inches'] = weather[weather.precipitation_inches.notnull()].precipitation_inches.median()
#print(weather.dtypes)

In [38]:
#Add weekend feature
training_data = training_data.merge(weather, how="left", on = ['date', 'city'])
training_data['weekend'] =pd.to_datetime(training_data['date']).dt.weekday>4


In [39]:
#change false/true to 1/0
bool_to_int={False:"0", True:"1"}
training_data.replace(bool_to_int, inplace=True)

In [40]:
#drop na values
training_data=training_data.dropna()

In [41]:
training_sf = training_data[training_data["city"]=="San Francisco"]
training_sj = training_data[training_data["city"]=="San Jose"]
training_mv = training_data[training_data["city"]=="Montain View"]
training_rc = training_data[training_data["city"]=="Redwood City"]
training_pa = training_data[training_data["city"]=="Palo Alto"]

In [42]:
training_sf = training_sf.drop(['date','city'], 1)
training_sj = training_sj.drop(['date','city'], 1)
training_mv = training_mv.drop(['date','city'], 1)
training_pa = training_pa.drop(['date','city'], 1)
training_rc = training_rc.drop(['date','city'], 1)

In [43]:
labels_sf = training_sf.trips
labels_sj = training_sf.trips
labels_mv = training_sf.trips
labels_pa = training_sf.trips
labels_rc = training_sf.trips

In [44]:
training_sf = training_sf.drop(['trips'], 1)
training_sj = training_sj.drop(['trips'], 1)
training_mv = training_mv.drop(['trips'], 1)
training_pa = training_pa.drop(['trips'], 1)
training_rc = training_rc.drop(['trips'], 1)


In [45]:
training_sf.to_csv('sf.csv')
training_sj.to_csv('sj.csv')
training_mv.to_csv('mv.csv')
training_pa.to_csv('pa.csv')
training_rc.to_csv('rc.csv')


In [46]:
labels_sf.to_csv('label_sf.csv')
labels_sj.to_csv('label_sj.csv')
labels_mv.to_csv('label_mv.csv')
labels_pa.to_csv('label_pa.csv')
labels_rc.to_csv('label_rc.csv')